# This is a chat bot model
<hr>

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()

True

to make the note book work synchoronized

In [3]:
import nest_asyncio
nest_asyncio.apply()

Get the OpenAI Key

In [4]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

Get the data

In [7]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('../data').load_data()

Chunk and split 

In [8]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [9]:
nodes

[TextNode(id_='1782dacb-11d5-44e9-9ee6-7f406e350ae1', embedding=None, metadata={'page_label': '1', 'file_name': 'Haida bracelet.pdf', 'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/llamaindex/adv_rag_system/../data/Haida bracelet.pdf', 'file_type': 'application/pdf', 'file_size': 227621, 'creation_date': '2024-09-29', 'last_modified_date': '2024-09-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='689e8bd5-a71a-4405-aebc-dec7f30703da', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Haida bracelet.pdf', 'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/llamaindex/adv_rag_system/../data/Haida bracelet.pdf', 'file_type': 'application/pdf'

Define the llm and the embedding method

In [10]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model='gpt-3.5-turbo')
Settings.embed_model = OpenAIEmbedding(model='text-embedding-3-small')

There are vector index and summary index

In [11]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

two different query engine

In [12]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)
vector_query_engine = vector_index.as_query_engine()

define the tool

In [13]:
from llama_index.core.tools import QueryEngineTool
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the documents"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for questions related to the content of the documents"
    ),
)

Selectors

In [14]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True,
)

Test model

In [15]:
from llama_index.core.response.pprint_utils import pprint_response

response = query_engine.query("What is the summary of the documents?")

print(response)
print("====================================================")
pprint_response(response)

Selecting query engine 0: The question specifically asks for a summary of the documents, which is related to summarization questions..
The documents provide information about a silver bracelet titled "Happy Negative Spaces" created by artist Robert Davidson in 1980. Davidson, a great grandson of Haida artist Charles Edenshaw, is known for combining innovation with traditional craftsmanship. The bracelet is considered one of his important early abstract works, showcasing his exploration of abstract possibilities in Haida form. The document also mentions an exhibition at the Seattle Art Museum in 2013 that featured Davidson's innovative work, including this bracelet, which later traveled to the National Museum of the American Indian in New York City.
Final Response: The documents provide information about a silver
bracelet titled "Happy Negative Spaces" created by artist Robert
Davidson in 1980. Davidson, a great grandson of Haida artist Charles
Edenshaw, is known for combining innovatio

Source node shows it go through the whole data source

In [22]:
print(len(response.source_nodes))

2


In [23]:
response = query_engine.query("What is the bracelet made of?")

print(response)
print("====================================================")
pprint_response(response)

Selecting query engine 1: The question is asking about the content of the documents, specifically what the bracelet is made of..
The bracelet is made of silver.
Final Response: The bracelet is made of silver.


In [24]:
print(len(response.source_nodes))

2
